<a href="https://colab.research.google.com/github/HSE-LAMBDA/MLatFIAN2020/blob/master/seminar01/MLatFIAN2020_seminar01_part2_SimpleMLFit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Let's do a Maximum Likelihood fit!


---


We'll need an extra package called `probfit` which is not available by default. It takes some time to install, so please run the installation command now:

In [ ]:
!pip install probfit

## What we'll need


---



*   ### Get and select the data
*   ### Define the Likelihood function
*   ### Maximize it (minimize negative Likelihood)
*   ### Plot the result



In [ ]:
# We'll store and handle our data with numpy.
#
# Also, we'll start by working with 'toy' data, i.e. we are
# going to randomly generate our data - we'll use numpy for
# that as well.
import numpy as np

# The plotting will be done in matplotlib:
import matplotlib.pyplot as plt
%matplotlib inline

## Toy data


---


In reality we don't usually detect just the decays we are looking for. Usually, there are also other processes that have the same signature as the process of interest, and there is no deterministic way to tell them apart.


Let's emulate this in our toy data!

In [ ]:
# Say, there's some particle with a mass of 125.18 GeV,
# which we can only measure with an uncertainty of 15 GeV:
signal = np.random.normal(loc=125.18, scale=15., size=10000)

# Then, imagine there are some continuous backgound processes
# with the same signature, which contribute to our particle
# mass measurement and are distributed as decaying exponent:
background = np.random.exponential(scale=80., size=90000)

# In real life there's no deterministic way to tell them apart,
# as they are 'shuffled' together:
data = np.concatenate([signal, background])
np.random.shuffle(data)

print("data.shape = {}".format(data.shape))
print("data = {}".format(data))

Now, `data` is a 1-dimentional array each element of which represents distinct mass measurements. Let's see what it looks like:

In [ ]:
plt.hist(data, bins=100, alpha=0.8);

OK, that's nice, but we probably don't need such a wide range. Let's only select the data from 10 to 200 GeV:

In [ ]:
# We'll define a tuple with our selection range, as
# this will come in handy later
mass_bound = (10., 200.)

data = data[(data > mass_bound[0]) & (data < mass_bound[1])]
plt.hist(data, bins=100, alpha=0.8);

## Defining the model PDF and Likelihood functions


---


OK, remember we installed something at the beginning of the section? It was `probfit` — a package that makes PDF and Likelihood definition really easy (see [this page](https://probfit.readthedocs.io/en/latest/index.html) for more details; they also have a good [tutorial here](http://nbviewer.ipython.org/urls/raw.github.com/scikit-hep/probfit/master/tutorial/tutorial.ipynb)). Let's import the package:

In [ ]:
import probfit

PDFs can just be python functions with the following signature:

`function_name(x, param1, param2, ...)`

However, one has to make sure they integrate up to 1:
$$\int_{\text{domain}}\text{PDF}(x)dx = 1$$

Since we've selected our data to be in the interval $(10, 200)~$GeV, we have to normalize our PDFs such that
$$\int_{10~\text{GeV}}^{200~\text{GeV}}\text{PDF}(x)dx = 1$$

As our data is a mixture of a peaking signal and a continuous background, we'll define the PDF in the following form:
$$\text{PDF}(x|m,\sigma,k, f_{\text{sig}}) = 
f_{\text{sig}}\cdot \mathscr{N}(x|m, \sigma) +
(1 - f_{\text{sig}})\cdot k e^{-kx},$$
where:


*   $x$ is the value of mass from an individual measurement
*   $m$ is the actual mass of the decaying particle
*   $sigma$ is detector resolution
*   $\mathscr{N}(x|m, \sigma)$ is normal (Gaussian) distribution
*   $k$ is the empirical background slope
*   $f_{\text{sig}}$ is the fraction of signal in the mixture



In [ ]:
# For the Gaussian we'll use the function already defined in
# probfit and just wrap it around our custom python function
# to rename the parameters
def SignalPDF(x, mass, sigma):
  return probfit.gaussian(x, mass, sigma)

# As for the exponential function, we'll define it ourselves
def BgPDF(x, exp_slope):
  return exp_slope * np.exp(-exp_slope * x)

# So far our functions are normalized differently:
#     SignalPDF is normalized for x in [-infinity, +infinity]
#     BgPDF is normalized for x in [0 GeV, +infinity]
# We need them both to be normalized for x in [10, 200] GeV.

# Luckily, probfit has a convinient 'Normalize' class for that:
SignalPDF_normed = probfit.Normalized(SignalPDF, mass_bound)
BgPDF_normed     = probfit.Normalized(BgPDF    , mass_bound)

# There's also a ready to use class to combine several PDFs in a sum:
ModelPDF = probfit.AddPdfNorm(SignalPDF_normed,
                              BgPDF_normed,
                              facname=['signal_fraction'])
# OK, our PDF is ready!

Now we have to define the Likelihood function for a set of measurements $\{x_i\}$:
$$\mathscr{L}(m,\sigma,k, f_{\text{sig}}) = \prod_i{\text{PDF}(x_i|m,\sigma,k, f_{\text{sig}})}$$

There's a ready to use class for that as well:

In [ ]:
unbinned_likelihood = probfit.UnbinnedLH(ModelPDF, data)

## Maximizing the Likelihood!

Actually, will **minimize** the  $-log(\mathscr{L})$, which has the same effect. In HEP it's a common practice to use [the MINUIT program](https://en.wikipedia.org/wiki/MINUIT) for minimization. It was originally written in 1970s in CERN and its Migrad algorithm has proven to be super robust and stable.

Although MINUIT is a C++ program (originally – FORTRAN), there is a python wrapper called `iminuit`. Let's import it:

In [ ]:
import iminuit

In order to run the minimization, one has to:


*   create an instance of `iminuit.Minuit` class
*   give it an instance to the function to be minimized
*   set up initial parameter values
*   set up parameter limits (optional, recommended)
*   set up initial steps for parameter variations (optional)

Firstly, let's see which parameters does our PDF have:

In [ ]:
print(iminuit.describe(unbinned_likelihood))

So, for the set of measurements $\{x_i\}$ our Likelihood will be a function of the following parameters:
```
mass
sigma
exp_slope
signal_fraction```

In [ ]:
# We'll define dictionaries of our parameter values.

# This will be our initial guess:
initial_par_values = dict(
  mass           =120.  ,
  sigma          = 25.  ,
  exp_slope      =  0.01,
  signal_fraction=  0.4 ,
)

# We'll define the limits:
limits = dict(
  limit_mass           =(50. , 200.),
  limit_sigma          =( 3. , 150.),
  limit_exp_slope      =(1e-4,   1.),
  limit_signal_fraction=( 0. ,   1.),
)

# And initial variation steps:
errors = dict(
  error_mass           =10.  ,
  error_sigma          =10.  ,
  error_exp_slope      =0.005,
  error_signal_fraction=0.2  ,
)

# Finally, let's create the minimizer:
minuit = iminuit.Minuit(
            unbinned_likelihood, **initial_par_values, **limits, **errors)


# N.B. If you're not familiar with the `func(**arg)` notation,
# the following two code snippets do the same thing:
#
#     arg = dict(x=1, y=2)
#     func(**arg)
#
# and
#
#     func(x=1, y=1)
#
# except that in the first case you also get a dictionary `arg` with the
# following content:
#
#     {'x' : 1, 'y' : 2}

OK, let's see how good our guess was:

In [ ]:
unbinned_likelihood.draw(minuit=minuit);

Not perfect. Now, let's run the minimization algorithm:

In [ ]:
minuit.migrad()

Nice! It also prints out the results. You want to make sure that the fit was successful (Valid == True), that none of your parameters has hit it's range limits, and also that the error matrix is positively defined (PosDef = True).

Let's run the accurate hessian estimating algorithm:

In [ ]:
minuit.hesse()

In [ ]:
# We've already used this function to plot our PDF against the data before.
# Now we've added a few parameters:
#   `parts=True` enables different components drawing, i.e. we'll see the
#                exponent and Gaussian independently
#   `parmloc` specifies where the fitted parameters printout will be located.
unbinned_likelihood.draw(minuit=minuit, parts=True, parmloc=(0.45, 0.95));

# Bonus part: speeding things up with Cython

[Cython](https://cython.org/) is an extension that allows you to convert python-like code to C, compile it and import the resulting objects back into your python session.

In [ ]:
!pip install Cython
%load_ext Cython

In [ ]:
%%cython --annotate

cimport cython
from libc.math cimport exp, M_PI, sqrt

@cython.binding(True)
def gauss_pdf_cython(double x, double mass, double sigma):
  return 1 / sqrt(2 * M_PI) / sigma * exp(-(x - mass)**2 / 2. / sigma**2)

@cython.binding(True)
def exp_pdf_cython(double x, double exp_slope):
  return exp(-exp_slope * x) * exp_slope

# See this for more info on binding:
# https://cython.readthedocs.io/en/latest/src/userguide/source_files_and_compilation.html#compiler-directives

In [ ]:
gauss_pdf_cython_normed = probfit.Normalized(gauss_pdf_cython, mass_bound)
exp_pdf_cython_normed   = probfit.Normalized(exp_pdf_cython  , mass_bound)


ModelPDF_cython = probfit.AddPdfNorm(gauss_pdf_cython_normed,
                                     exp_pdf_cython_normed,
                                     facname=['signal_fraction'])
print(iminuit.describe(ModelPDF_cython))

In [ ]:
unbinned_likelihood_cython = probfit.UnbinnedLH(ModelPDF_cython, data)

minuit_cython = iminuit.Minuit(
                    unbinned_likelihood_cython,
                    **initial_par_values, **limits, **errors)

unbinned_likelihood_cython.draw(minuit=minuit_cython, parts=True, parmloc=(0.45, 0.95));

In [ ]:
%timeit -n1 -r1 minuit_cython.migrad();

In [ ]:
unbinned_likelihood_cython.draw(minuit=minuit_cython, parts=True, parmloc=(0.45, 0.95));

# Bonus part 2: pure `iminuit` without `probfit`

Probfit is great but it's limited to a single observalbe. Here we'll demonstrate a maximum likelihood fit with pure `iminuit` when there's two observables.

Our true data will be distributed as
$$y = 2\cdot x^2 - x + \text{noise}.$$

The two observables will be:
$$x_1 \equiv x$$
$$x_2 \equiv x^2$$

and our model is:
$$p\left(y|x, w_1, w_2, \sigma\right)
=\mathscr{N}(y|\mu(x), \sigma^2)$$
$$\mu(x)=w_1\cdot x + w_2\cdot x^2$$

In [ ]:
N = 200

x = np.random.uniform(size=N)
X = np.stack([
    x, x**2
], axis=1)

y = np.dot([-1., 2.], X.T)
y += np.random.normal(size=y.shape) * 0.1

print(X.shape, y.shape)

plt.scatter(x, y);

In [ ]:
def PDF(y, X, w1, w2, sigma):
  return (1. / np.sqrt(2 * np.pi) / sigma) * np.exp(
      -(y - np.dot([w1, w2], X.T))**2 / (2 * sigma**2)
  )

def NLL(w1, w2, sigma):
  return -np.log(PDF(y, X, w1, w2, sigma)).sum()

In [ ]:
iminuit.describe(NLL)

In [ ]:
minuit = iminuit.Minuit(
    NLL,
    w1=0., w2=0., sigma=1.0,
    limit_sigma=(0.001, 10.),
    limit_w1=(-1000., 1000.),
    limit_w2=(-1000., 1000.),
    error_w1=1., error_w2=1., error_sigma=0.1,
)

In [ ]:
minuit.migrad()

In [ ]:
minuit.hesse()

In [ ]:
minuit.draw_contour("w1", "w2");